In [1]:
import torch
data = torch.load("cartwheel copy.pt")
data = data[80:, :]
# data = torch.concat((data, torch.zeros(160, 5)), dim=1)
data.shape

torch.Size([80, 35])

In [2]:
data

tensor([[-0.2069,  0.9914,  0.9110,  ...,  0.1546,  0.1087,  0.0074],
        [-0.2190,  1.0067,  0.9109,  ...,  0.1611,  0.1134,  0.0062],
        [-0.2299,  1.0208,  0.9102,  ...,  0.1455,  0.1227, -0.0051],
        ...,
        [-0.2466,  2.0540,  0.8465,  ...,  0.0592, -0.2009,  0.2332],
        [-0.2435,  2.0658,  0.8467,  ...,  0.0733, -0.2067,  0.2150],
        [-0.2394,  2.0782,  0.8469,  ...,  0.0850, -0.2133,  0.1992]])

In [3]:
min_val, max_val = data.min(), data.max()

In [4]:
normalized_data = (data - min_val) / (max_val - min_val)

In [5]:
flattened_data = normalized_data.flatten().unsqueeze(0)
flattened_data.shape

torch.Size([1, 2800])

In [7]:
train = flattened_data.unsqueeze(0)
train.shape

torch.Size([1, 1, 2800])

In [8]:
cuda_available = torch.cuda.is_available()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
from denoising_diffusion_pytorch import (
    Unet1D,
    GaussianDiffusion1D,
    Trainer1D,
    Dataset1D,
)

model = Unet1D(dim=64, dim_mults=(1, 2, 4, 8), channels=1)
model = model.to(device)
train = train.to(device)

diffusion = GaussianDiffusion1D(
    model, seq_length=2800, timesteps=100, objective="pred_v"
)

/home/kenji/mambaforge/envs/ddpm/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
dataset = Dataset1D(train)

trainer = Trainer1D(
    diffusion,
    dataset=dataset,
    train_batch_size=32,
    train_lr=8e-5,
    train_num_steps=10000,  # total training steps
    gradient_accumulate_every=2,  # gradient accumulation steps
    ema_decay=0.995,  # exponential moving average decay
    amp=True,  # turn on mixed precision
)
trainer.train()

/home/kenji/mambaforge/envs/ddpm/lib/python3.12/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(split_batches=True)
  warnings.warn(
loss: 0.0204:  30%|███       | 3041/10000 [06:54<15:49,  7.33it/s]  
/home/kenji/mambaforge/envs/ddpm/lib/python3.12/site-packages/einops/einops.py:827: SyntaxWarning: invalid escape sequence '\s'
  """


KeyboardInterrupt: 

In [11]:
sampled_seq = diffusion.sample(batch_size=1)

sampling loop time step: 100%|██████████| 100/100 [00:02<00:00, 43.66it/s]


In [12]:
sampled_seq[0]

tensor([[0.4862, 0.6790, 0.6248,  ..., 0.4955, 0.4462, 0.5188]],
       device='cuda:0')

In [13]:
denormalized_sample = sampled_seq[0] * (max_val - min_val) + min_val
denormalized_sample

tensor([[-0.0355,  1.0547,  0.7484,  ...,  0.0174, -0.2613,  0.1492]],
       device='cuda:0')

In [14]:
denormalized_sample.shape

torch.Size([1, 2800])

In [15]:
denormalized_sample = denormalized_sample.reshape(80, 35)
denormalized_sample.shape

torch.Size([80, 35])

In [16]:
denormalized_sample[0]

tensor([-0.0355,  1.0547,  0.7484, -0.0837, -1.0019, -0.1508,  0.1947, -0.2580,
        -0.3069, -0.2202,  0.0051,  0.2620, -0.2746, -2.4628, -0.7812, -1.5687,
         0.2500,  2.5898, -0.7859,  1.3255,  0.0814, -0.5834, -0.7522, -0.5622,
        -0.9059,  0.3070,  0.6158,  0.1634,  1.1963, -0.8728,  0.9382, -0.8224,
         0.0346,  0.0123, -0.1380], device='cuda:0')

In [17]:
denormalized_sample = denormalized_sample.cpu().detach().numpy()
denormalized_sample.shape

(80, 35)

In [18]:
import numpy as np
np.save("cartwheel-half-2.npy", denormalized_sample)